In [1]:
# Required installations
# !pip cache purge
# !python3 -m pip install -U scikit-learn scipy
# !pip install nltk
# !pip install keras
# !pip install gensim
# !pip install matplotlib
# !pip install pandas
# !pip install scikit-learn
# !pip install nltk
# !pip install tensorflow

In [2]:
# Required imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import re
import pickle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, accuracy_score
from nltk.corpus import stopwords
from imblearn.over_sampling import SMOTE
from collections import Counter

stop_words = set(stopwords.words('english'))

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from keras.layers import Embedding
from keras.layers import Dense, Input, GlobalMaxPooling1D, GlobalAveragePooling2D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv2D, MaxPooling2D, Dropout,concatenate
from keras.layers.core import Reshape, Flatten
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import SGD, RMSprop, Adagrad, Adadelta, Adam, Adamax, Nadam
from keras.models import Model
from keras import regularizers

import gensim
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models.keyedvectors import KeyedVectors

In [3]:
# cleaning data
def clean_post(post):
    post = post.lower()
    post = re.sub(r"\n", " ", post)
    post = re.sub("[\<\[].*?[\>\]]", " ", post)
    post = re.sub(r"[^a-z ]", " ", post)
    post = re.sub(r"\b\w{1,3}\b", " ", post)
    return " ".join([x for x in post.split() if x not in stop_words])

In [4]:
# Different techniques for tackling class imbalance
from imblearn.under_sampling import RandomUnderSampler, TomekLinks, NearMiss, ClusterCentroids
from imblearn.over_sampling import RandomOverSampler, SMOTE

def balance_data(x, y, _type):
    if _type == 0:
        ros = RandomOverSampler(random_state=42)
        return ros.fit_resample(x, y)
    elif _type == 1:
        rus = RandomUnderSampler(random_state=42, replacement=True)
        return rus.fit_resample(x, y)
    elif _type == 2:
        smote = SMOTE()
        return smote.fit_resample(x, y)
    elif _type == 3:
        nm = NearMiss()
        return nm.fit_resample(x, y)
    elif _type == 5:
        cc = ClusterCentroids()
        return cc.fit_resample(x, y)
    elif _type == 6:
        tl = TomekLinks()
        return tl.fit_resample(x, y)
    return x, y
    # Another technique is penalizing the algo with class_weight=balanced, using stratified cross validation

In [5]:
# Load data
data = pd.read_csv('../split_data/train_and_valid.csv')
data = shuffle(data)

# Class split stats
print(data.groupby(['mental_disorder'])[['mental_disorder']].describe())
X = data['post'].apply(lambda post: clean_post(post))
label_encoder = LabelEncoder()
y1 = label_encoder.fit_transform(np.array(data['mental_disorder']))
y = to_categorical(y1)

# 70-15-15 split (test data is unseen)
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.176, random_state=321)

                mental_disorder                             
                          count unique            top   freq
mental_disorder                                             
EDAnonymous               12339      1    EDAnonymous  12339
addiction                  6515      1      addiction   6515
adhd                      38786      1           adhd  38786
alcoholism                 5026      1     alcoholism   5026
anxiety                   48971      1        anxiety  48971
autism                     7583      1         autism   7583
bipolarreddit              4929      1  bipolarreddit   4929
bpd                       20606      1            bpd  20606
depression                99809      1     depression  99809
healthanxiety              7373      1  healthanxiety   7373
lonely                    20103      1         lonely  20103
ptsd                       7336      1           ptsd   7336
schizophrenia              7351      1  schizophrenia   7351
socialanxiety           

In [6]:
# Using keras tokenizer on text for pre-processing
MAX_WORDS_LIMIT = 30000
tokenizer = Tokenizer(num_words=MAX_WORDS_LIMIT, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n\'', lower=True)
tokenizer.fit_on_texts(X_train)
with open('../models/tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f, protocol=pickle.HIGHEST_PROTOCOL)
word_index = tokenizer.word_index
print(f'Unique tokens found: {len(word_index)}')

Unique tokens found: 110301


In [7]:
# Convert  texts to sequence of integers
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_valid = tokenizer.texts_to_sequences(X_valid)

# Limit size of train/validation/test sequences to 200 and pad the sequence
X_train = pad_sequences(sequences_train, maxlen=200)
X_valid = pad_sequences(sequences_valid, maxlen=X_train.shape[1])
print(f'Shape of X train tensor: {X_train.shape}, X validation tensor: {X_valid.shape}')

# Convert target to array
y_train, y_valid = np.asarray(y_train), np.asarray(y_valid)
print(f'Shape of y train tensor: {y_train.shape}, y validation tensor: {y_valid.shape}')

Shape of X train tensor: (298700, 200), X validation tensor: (63800, 200)
Shape of y train tensor: (298700, 15), y validation tensor: (63800, 15)


In [8]:
# Creating word embeddings via pre-trained word2vec model
WORD_EMBEDDING_DIM = 300
word_vectors = KeyedVectors.load_word2vec_format('../reddit_mental_health_dataset/GoogleNews-vectors-negative300.bin', binary=True)
embeddings_matrix = np.zeros((MAX_WORDS_LIMIT, WORD_EMBEDDING_DIM))

# Computing embeddings matrix and embedding layer
for word, i in word_index.items():
    if i >= MAX_WORDS_LIMIT:
        break
    try:
        embeddings_matrix[i] = word_vectors[word]
    except:
        embeddings_matrix[i] = np.zeros(WORD_EMBEDDING_DIM)
embedding_layer = Embedding(MAX_WORDS_LIMIT, WORD_EMBEDDING_DIM, weights=[embeddings_matrix], trainable=True)

In [9]:
# # Approach - 1
# # Parameters
# sequence_length = X_train.shape[1]
# filter_sizes = [3, 4]
# num_filters = 128
# drop = 0.4

# # Obtaining embeddings based on input sequence
# inputs = Input(shape=(sequence_length,))
# embedding = embedding_layer(inputs)
# reshape = Reshape((sequence_length, WORD_EMBEDDING_DIM, 1))(embedding)

# # Creating convolutional and maxpool layers
# conv_layers, maxpool_layers = [], []
# for i in range(2):
#     conv_layers.append(Conv2D(num_filters, (filter_sizes[i], WORD_EMBEDDING_DIM), activation='relu', 
#                               kernel_regularizer=regularizers.l2(0.01))(reshape))
#     maxpool_layers.append(MaxPooling2D((sequence_length - filter_sizes[i] + 1, 1), strides=(1, 1))(conv_layers[i]))

# # Constructing the complete network and creating model
# merged_tensor = concatenate(maxpool_layers, axis=1)
# flatten = Flatten()(merged_tensor)
# reshape = Reshape((2*num_filters,))(flatten)
# dropout = Dropout(drop)(flatten)
# conc = Dense(60)(flatten)
# output = Dense(units=15, activation='softmax')(conc)
# model = Model(inputs, output)

In [10]:
# # Approach - 2
# # Parameters
# sequence_length = X_train.shape[1]
# num_filters = 128
# drop = 0.25

# # Obtaining embeddings based on input sequence
# inputs = Input(shape=(sequence_length,))
# embedding = embedding_layer(inputs)
# reshape = Reshape((sequence_length, WORD_EMBEDDING_DIM, 1))(embedding)

# # Constructing the complete network and creating model
# conv = Conv2D(num_filters, (5, WORD_EMBEDDING_DIM), activation='relu', kernel_regularizer=regularizers.l2(0.01))(reshape)
# dropout = Dropout(drop)(conv)
# pool = MaxPooling2D((sequence_length - 4, 1), strides=(1, 1))(dropout)
# flatten = GlobalAveragePooling2D()(pool)
# dropout = Dropout(drop)(flatten)
# conc = Dense(60)(flatten)
# output = Dense(units=15, activation='softmax')(conc)
# model = Model(inputs, output)

In [11]:
# Approach 3
# Parameters
sequence_length = X_train.shape[1]
num_filters = 250
drop = 0.25

# Obtaining embeddings based on input sequence
inputs = Input(shape=(sequence_length,))
embedding = embedding_layer(inputs)

# Constructing the complete network and creating model
conv = Conv1D(num_filters, 5, activation='relu', kernel_regularizer=regularizers.l2(0.01))(embedding)
dropout = Dropout(drop)(conv)
pool = GlobalMaxPooling1D()(dropout)
conc = Dense(100)(pool)
dropout2 = Dropout(drop)(conc)
output = Dense(units=15, activation='softmax')(dropout2)

2021-11-05 12:37:16.175112: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcusolver.so.11'; dlerror: libcusolver.so.11: cannot open shared object file: No such file or directory
2021-11-05 12:37:16.523128: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2021-11-05 12:37:16.523189: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1835] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2021-11-05 12:37:16.524340: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Libra

In [12]:
# Fitting Model to the data
X_tr, y_tr = X_train, y_train
for _type in [1, 3, 5, 6, -1, 0, 2]:
    model = Model(inputs, output)
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=1e-3), metrics=["accuracy"])
    print('#'*110)
    print()
    if _type == -1:
        print('Without any oversampling/undersampling')
    else:
        print(f'With sampling type: {_type}')
    print()
    print()
    X_train, y_train = balance_data(X_tr, y_tr, _type)
    hist_adam = model.fit(X_train, y_train, batch_size=600, epochs=7, verbose=2, 
                          validation_data=(X_valid, y_valid))
    # Saving model
    model.save(f'../models/CNN_model_{_type}')

##############################################################################################################

With sampling type: 1




2021-11-05 12:37:17.264080: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/7


101/101 - 87s - loss: 2.5126 - accuracy: 0.5333 - val_loss: 1.6889 - val_accuracy: 0.5629


Epoch 2/7


...

Epoch 6/7


2055/2055 - 1396s - loss: 2.3831 - accuracy: 0.2848 - val_loss: 1.5411 - val_accuracy: 0.6123


Epoch 7/7


2055/2055 - 1396s - loss: 2.3733 - accuracy: 0.2882 - val_loss: 1.5623 - val_accuracy: 0.6096


INFO:tensorflow:Assets written to: ../models/CNN_model_2/assets
